<a href="https://colab.research.google.com/github/rboghe/cened/blob/master/cened_merged_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TO DO



*   take care of _x and _y


##Loading Files

In [1150]:
import os
import urllib.request
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, make_scorer
from lightgbm import LGBMRegressor, plot_importance
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV
import seaborn as sns

# Silence panda's setting with copy warning
pd.options.mode.chained_assignment = None

# Set Seaborn style
sns.set_theme(style="white")

In [1151]:
url = 'https://www.dati.lombardia.it/api/views/bbky-sde5/rows.csv?accessType=DOWNLOAD'

if os.path.isdir('/tmp'):
    if os.path.isfile('/tmp/cened20.csv'):
      pass
    else:
      urllib.request.urlretrieve(url, '/tmp/cened20.csv')
else:
  os.mkdir('/tmp')
  urllib.request.urlretrieve(url, '/tmp/cened20.csv')

In [1152]:
url = 'https://www.dati.lombardia.it/api/views/rsg3-xhvk/rows.csv?accessType=DOWNLOAD'


if os.path.isfile('/tmp/cened12.csv'):
  pass
else:
  urllib.request.urlretrieve(url, '/tmp/cened12.csv')

In [1153]:
url = 'https://raw.githubusercontent.com/rboghe/cened/master/degreedays.txt'

urllib.request.urlretrieve(url, '/tmp/ddh.csv')

ddh = pd.read_csv('/tmp/ddh.csv', usecols = ['comune','dd'])

In [1154]:
cols20 = ['RIQUALIFICAZIONE_ENERGETICA',
           'ANNO_COSTRUZIONE','SUPERF_UTILE_RISCALDATA', 
           'VOLUME_LORDO_RISCALDATO', 'SUPERFICIE_DISPERDENTE', 'RAPPORTO_SV','A_SOL_EST_A_SUP_UTILE','Y_IE', 
           'CLASSIFICAZIONE_DPR', 'COMUNE', 'EP_H_ND','EP_GL_NREN','EP_GL_REN', 
           'NUMERO_UNITA_IMMOBILIARI', 'RISTRUTTURAZIONE_IMPORTANTE', 
           'COMUNE','FOGLIO', 'PARTICELLA', 'SUBALTERNO']

cols12 = ['COMUNE','FOGLIO', 'PARTICELLA', 'SUBALTERNO','SUPERFICIE_DISPERDENTE',
          'TRASMITTANZA_MEDIA_COPERTURA','TRASMITTANZA_MEDIA_INVOLUCRO',
          'TRASMITTANZA_MEDIA_BASAMENTO','TRASMITTANZA_MEDIA_SERRAMENTO',
          'SUPERFICIE_LORDA', 'VOLUME_LORDO', 'DESTINAZIONE_DI_USO',
          'SUPERFICIE_VETRATA_OPACA', 'VOLUME_NETTO',
          'SUPERFICIE_NETTA', 'ANNO_COSTRUZIONE','NUMERO_RICAMBI_ORARI', 'EPH']
  
cened20 = pd.read_csv('/tmp/cened20.csv', usecols = cols20)
cened12 = pd.read_csv('/tmp/cened12.csv', usecols = cols12)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6,7,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Pre-processing

In [1155]:
cened20['EP_GL_NREN'].fillna(0, inplace=True)
cened20['EP_GL_REN'].fillna(0, inplace=True)

# Drop NaN 
cened20 = cened20.dropna()
cened12 = cened12.dropna() 

In [1156]:
# Only entire buildings
# cened20 = cened20[cened20['INTERO_EDIFICIO'] == True]

In [1157]:
# Drop renovations
cened20 = cened20[cened20['RISTRUTTURAZIONE_IMPORTANTE'] == False]
cened20 = cened20[cened20['RIQUALIFICAZIONE_ENERGETICA'] == False]

In [1158]:
cened20.ANNO_COSTRUZIONE = cened20.ANNO_COSTRUZIONE.astype(str)
cened20.ANNO_COSTRUZIONE = cened20.ANNO_COSTRUZIONE.map(lambda x:x[-4:])
cened20.ANNO_COSTRUZIONE = cened20.ANNO_COSTRUZIONE.astype(int)

cened20['PERIODO_20'] = pd.cut(cened20.ANNO_COSTRUZIONE, bins=[0,1930,1945,1960,1976,1992,2006,3000],
     labels=['Before 1930','1930-1945','1946-1960','1961-1976', '1977-1992', '1993-2006','After 2006']).cat.codes

In [1159]:
cened12.ANNO_COSTRUZIONE = cened12.ANNO_COSTRUZIONE.astype(str)
cened12.ANNO_COSTRUZIONE = cened12.ANNO_COSTRUZIONE.map(lambda x:x[-4:])
cened12.ANNO_COSTRUZIONE = cened12.ANNO_COSTRUZIONE.astype(int)

cened12['PERIODO_12'] = pd.cut(cened12.ANNO_COSTRUZIONE, bins=[0,1930,1945,1960,1976,1992,2006,3000],
     labels=['Before 1930','1930-1945','1946-1960','1961-1976', '1977-1992', '1993-2006','After 2006']).cat.codes

## Merge

In [1160]:
# Take care of city names
cened12['COMUNE'] = cened12['COMUNE'].str.lower()
cened12['COMUNE'] = cened12['COMUNE'].str.replace(r"o`",  "o'")
cened12['COMUNE'] = cened12['COMUNE'].str.replace(r"baranzate",  "bollate")
cened12['COMUNE'] = cened12['COMUNE'].str.replace(r"bovisio masciago",  "bovisio-masciago")
cened12['COMUNE'] = cened12['COMUNE'].str.replace(r"cornate d`adda",  "cornate d'adda")
cened12['COMUNE'] = cened12['COMUNE'].str.replace(r"cortenuova",  "cortenova")
cened12['COMUNE'] = cened12['COMUNE'].str.replace(r"sant'omobono terme",  "sant'omobono imagna")

cened20['COMUNE'] = cened20['COMUNE'].str.lower()
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"`",  "'")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"o`",  "o'")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"baranzate",  "bollate")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"bovisio masciago",  "bovisio-masciago")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"cornate d`adda",  "cornate d'adda")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"cortenuova",  "cortenova")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"sant'omobono terme",  "sant'omobono imagna")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"borgo virgilio",  "virgilio")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r" - ",  "-")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"cassina de pecchi",  "cassina de' pecchi")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"cerano intelvi",  "cerano d'intelvi")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"colverde",  "drezzo")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"cornale e bastida",  "cornale")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"corteolona e genzone",  "corteolona")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"costa serina",  "costa di serina")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"gabbioneta binanuova",  "gabbioneta-binanuova")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"gadesco pieve delmona",  "gadesco-pieve delmona")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"gerre de'caprioli",  "gerre de' caprioli")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"gravedona ed uniti",  "grandola ed uniti")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"la valletta brianza",  "perego")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"lonato del garda",  "lonato")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"maccagno con pino e veddasca",  "maccagno")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"ramponio vernia",  "ramponio verna")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"sermide e felonica",  "sermide")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"toscolano maderno",  "toscolano-maderno")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"tremezzina",  "tremezzo")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"val brembilla",  "brembilla")
cened20['COMUNE'] = cened20['COMUNE'].str.replace(r"verderio",  "verderio inferiore")

In [1161]:
exall12 = cened12['SUBALTERNO'].str.extractall('(\d+)').fillna(0)
cened12 = exall12.join(cened12.reindex(exall12.index, level=0)).reset_index()
cened12['SUBALTERNO'] = cened12[0]

exall20 = cened20['SUBALTERNO'].str.extractall('(\d+)').fillna(0)
cened20 = exall20.join(cened20.reindex(exall20.index, level=0)).reset_index()
cened20['SUBALTERNO'] = cened20[0]

In [1162]:
# Merge dfs
cenedtot = pd.merge(left=cened20, right=cened12, left_on=['COMUNE','FOGLIO', 'PARTICELLA', 'SUBALTERNO'], #, 'PERIODO' 
                    right_on=['COMUNE','FOGLIO', 'PARTICELLA', 'SUBALTERNO'],  how='inner') # , 'SUBALTERNO' | , 'SUBALTERNO'

In [1163]:
print(len(cenedtot))

2287


## Pre-filtering

In [1164]:
# Compute difference between ext surfaces
# cenedtot['surfdiff'] = abs((cenedtot['SUPERF_UTILE_RISCALDATA'] - cenedtot['SUPERFICIE_NETTA'])/cenedtot['SUPERF_UTILE_RISCALDATA'])

# Drop rows with high difference
#cenedtot = cenedtot[cenedtot['surfdiff'] < 0.25]

# Compute difference between ext surfaces
#cenedtot['surfdiff'] = abs((cenedtot['SUPERFICIE_DISPERDENTE_y'] - cenedtot['SUPERFICIE_DISPERDENTE_x'])/cenedtot['SUPERFICIE_DISPERDENTE_y'])

# Drop rows with high difference
#cenedtot = cenedtot[cenedtot['surfdiff'] < 0.25]

In [1165]:
# Compute difference between EPH
#cenedtot['ephdiff'] = abs((cenedtot['EPH'] - cenedtot['EP_H_ND'])/cenedtot['EPH'])

# Drop rows with high difference
#cenedtot = cenedtot[cenedtot['ephdiff'] < 0.30]

In [1166]:
cenedtot['DESTINAZIONE_DI_USO'] = cenedtot['DESTINAZIONE_DI_USO'].str.replace(r'E.1 (1)', 'E.1(1)')
cenedtot = cenedtot[cenedtot['DESTINAZIONE_DI_USO'] == 'E.1(1)']
# cenedtot = cenedtot[cenedtot['DESTINAZIONE_DI_USO'].str.contains('|'.join(['E.1', 'E.2']))]

In [1167]:
cenedtot = cenedtot.dropna() 

# Preprocessing

In [1168]:
cenedtot['SUPERFICIE_DISPERDENTE'] = (cenedtot['SUPERFICIE_DISPERDENTE_x'] + cenedtot['SUPERFICIE_DISPERDENTE_y'])/2

In [1169]:
# Correct floats
for col in ['SUPERF_UTILE_RISCALDATA',
           'VOLUME_LORDO_RISCALDATO', 'SUPERFICIE_DISPERDENTE',
           'RAPPORTO_SV','A_SOL_EST_A_SUP_UTILE','Y_IE','EP_H_ND','EP_GL_NREN','EP_GL_REN']:
    cenedtot[col] = cenedtot[col].astype(str)
    cenedtot[col] = cenedtot[col].str.replace(r',', '')
    cenedtot[col] = cenedtot[col].astype("float")

In [1170]:
# Take care of ach
cenedtot.NUMERO_RICAMBI_ORARI = cenedtot.NUMERO_RICAMBI_ORARI.astype(str)
cenedtot.NUMERO_RICAMBI_ORARI = cenedtot.NUMERO_RICAMBI_ORARI.map(lambda x:x[:6])
cenedtot.NUMERO_RICAMBI_ORARI = cenedtot.NUMERO_RICAMBI_ORARI.astype(float)

##Feature engineering

In [1171]:
# Add ddh
ddh['comune'] = ddh['comune'].str.lower()
ddh['comune'] = ddh['comune'].str.replace(r"è",  "e'")
ddh['comune'] = ddh['comune'].str.replace(r"è",  "e'")
ddh['comune'] = ddh['comune'].str.replace(r"é",  "e'")
ddh['comune'] = ddh['comune'].str.replace(r"ò",  "o'")
ddh['comune'] = ddh['comune'].str.replace(r"ù",  "u'")

cenedtot = cenedtot.merge(ddh, left_on='COMUNE', right_on='comune', how='left')

In [1172]:
# Opaque surface
cenedtot['SUP_OPACA'] = cenedtot['SUPERFICIE_DISPERDENTE']/(1+cenedtot['SUPERFICIE_VETRATA_OPACA'])

# Glazing
cenedtot['SUP_FINESTRATA'] = cenedtot['SUPERFICIE_DISPERDENTE'] - cenedtot['SUP_OPACA']

In [1173]:
cenedtot['ALTEZZA_MEDIA_12'] = cenedtot['VOLUME_NETTO']/cenedtot['SUPERFICIE_NETTA']
cenedtot['ALTEZZA_MEDIA_20'] = cenedtot['VOLUME_LORDO_RISCALDATO']/cenedtot['SUPERF_UTILE_RISCALDATA']

In [1174]:
cenedtot['TRASMITTANZA_MEDIA'] = (cenedtot['TRASMITTANZA_MEDIA_SERRAMENTO'] + cenedtot['TRASMITTANZA_MEDIA_COPERTURA'] +
         cenedtot['TRASMITTANZA_MEDIA_BASAMENTO'] + cenedtot['TRASMITTANZA_MEDIA_INVOLUCRO'])/4

In [1175]:
cenedtot['EP_GL_TOT'] = cenedtot['EP_GL_REN'] + cenedtot['EP_GL_NREN']

In [1176]:
# East facing surface
cenedtot['A_SOL_EST'] = cenedtot['A_SOL_EST_A_SUP_UTILE']*cenedtot['SUPERF_UTILE_RISCALDATA']

##Filters

In [1177]:
for col in ['TRASMITTANZA_MEDIA_COPERTURA','TRASMITTANZA_MEDIA_INVOLUCRO',
          'TRASMITTANZA_MEDIA_BASAMENTO','TRASMITTANZA_MEDIA_SERRAMENTO']:
          cenedtot = cenedtot[cenedtot[col].between(0,6, inclusive=False)]

cenedtot = cenedtot[cenedtot['TRASMITTANZA_MEDIA'] > 0.15]
cenedtot = cenedtot[cenedtot['TRASMITTANZA_MEDIA'] < 4]

cenedtot = cenedtot[cenedtot['Y_IE'] <= 6]
cenedtot = cenedtot[cenedtot['Y_IE'] >= 0.01]

In [1178]:
# Drop buildings with wrong An
cenedtot = cenedtot[cenedtot['SUPERFICIE_NETTA'] >= 50]
cenedtot = cenedtot[cenedtot['SUPERF_UTILE_RISCALDATA'] >= 50]

# Drop buildings with wrong Vn
cenedtot = cenedtot[cenedtot['VOLUME_NETTO'] > 150]
cenedtot = cenedtot[cenedtot['VOLUME_LORDO_RISCALDATO'] >= 150]

In [1179]:
# Glazed surface
cenedtot = cenedtot[cenedtot['SUP_FINESTRATA'] > 1]

# Glazing ratio
cenedtot = cenedtot[cenedtot['SUPERFICIE_VETRATA_OPACA'] > 0]
cenedtot = cenedtot[cenedtot['SUPERFICIE_VETRATA_OPACA'] < 0.9]

In [1180]:
cenedtot = cenedtot[cenedtot['ALTEZZA_MEDIA_12'] > 2.4]
cenedtot = cenedtot[cenedtot['ALTEZZA_MEDIA_20'] > 2.4]

In [1181]:
# Drop buildings with wrong EPH
cenedtot = cenedtot[cenedtot['EP_H_ND'] > 5]
cenedtot = cenedtot[cenedtot['EP_H_ND'] < 1000]

In [1182]:
cenedtot = cenedtot[cenedtot['A_SOL_EST_A_SUP_UTILE'] <= 0.5]
cenedtot = cenedtot[cenedtot['A_SOL_EST_A_SUP_UTILE'] > 0]

In [1183]:
len(cenedtot)

1341

# Shuffle

In [1184]:
cened = cenedtot.sample(frac=1, random_state = 42).reset_index(drop=True)

# Reserve a test set

In [1185]:
msk = np.random.rand(len(cened)) < 0.8
train = cened[msk]
test = cened[~msk]

In [1186]:
print(len(train))

1062


In [1187]:
print(len(test))

279


# Define MAPE

In [1188]:
def neg_mape(y_true, y_pred):
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)
  return -np.abs((y_true - y_pred)/y_true).mean()

In [1189]:
def modified_neg_mape(y_true, y_pred):
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)
  errors = np.abs((y_true - y_pred)/y_true)
  index = np.argwhere(errors > 1)
  mod_errors = np.delete(errors, index)
  return -mod_errors.mean()

In [1190]:
neg_mape_scorer = make_scorer(neg_mape, greater_is_better=True)

In [1191]:
modified_neg_mape_scorer = make_scorer(modified_neg_mape, greater_is_better=True)

# 2.0 inputs

## Random search

In [1192]:
mlcol = ['SUPERF_UTILE_RISCALDATA',
           'VOLUME_LORDO_RISCALDATO',
           'SUPERFICIE_DISPERDENTE_x', 'dd','PERIODO_20',
           'RAPPORTO_SV','A_SOL_EST_A_SUP_UTILE','Y_IE',
           'ALTEZZA_MEDIA_20', 'A_SOL_EST', 'NUMERO_UNITA_IMMOBILIARI']

In [1193]:
cat_feats = ['PERIODO_20']

In [1194]:
param_dist = {'feature_fraction': np.linspace(0.4, 1, num=7),
              'num_leaf' : list(range(20,40)),
             'max_depth' : [-1],
             'max_bin' : [100, 200, 300, 500, 750, 1000, 2000],
             'bagging_fraction' : np.linspace(0.4, 1, num=14),
             'bagging_freq' : list(range(1,10)),
             'lambda_l1' : stats.uniform(0, 0.6),
             'lambda_l2' : stats.uniform(0, 0.6)}

lgbm = LGBMRegressor(n_estimators = 200, silent = True, verbose = 0, is_training_metric = True, n_jobs = 1, 
                     eval_metric  = 'mape')

n_iter_search = 100

random_search = RandomizedSearchCV(lgbm, param_distributions=param_dist, n_iter=n_iter_search, 
                                   scoring=neg_mape_scorer, cv = 5, n_jobs = 6, verbose = 2)

random_search.fit(cened[mlcol], cened['EP_H_ND'], categorical_feature = cat_feats)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    8.3s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:   29.2s
[Parallel(n_jobs=6)]: Done 353 tasks      | elapsed:  1.0min
[Parallel(n_jobs=6)]: Done 500 out of 500 | elapsed:  1.4min finished
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['PERIODO_20']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=LGBMRegressor(boosting_type='gbdt',
                                           class_weight=None,
                                           colsample_bytree=1.0,
                                           eval_metric='mape',
                                           importance_type='split',
                                           is_training_metric=True,
                                           learning_rate=0.1, max_depth=-1,
                                           min_child_samples=20,
                                           min_child_weight=0.001,
                                           min_split_gain=0.0, n_estimators=200,
                                           n_jobs=1, num_leaves=31,
                                           objective=None, random_sta...
                                        'lambda_l1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fcd6964f940>,
            

## Print results

In [1195]:
print(random_search.best_params_)

{'bagging_fraction': 1.0, 'bagging_freq': 4, 'feature_fraction': 0.7, 'lambda_l1': 0.25485420011261123, 'lambda_l2': 0.038225498964210035, 'max_bin': 200, 'max_depth': -1, 'num_leaf': 28}


In [1196]:
print(random_search.best_score_)

-0.3305996877778375


In [1197]:
# Create a results df
cv_df = pd.DataFrame(random_search.cv_results_)
cv_df = cv_df.sort_values(by = ['rank_test_score']).reset_index()

# Filter columns
res_cols = [col for col in cv_df if col.startswith('split')]

# Select best iter
best_res = cv_df.loc[0]

# Print results for each fold
print(best_res[res_cols])

split0_test_score   -0.293085
split1_test_score   -0.334908
split2_test_score   -0.370681
split3_test_score   -0.289944
split4_test_score   -0.364381
Name: 0, dtype: object


## Train the final model

In [1198]:
light = LGBMRegressor(n_estimators = 1000, silent = False, verbose = 2, is_training_metric = True, n_jobs = 6)
light.set_params(**random_search.best_params_)

LGBMRegressor(bagging_fraction=1.0, bagging_freq=4, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.7,
              importance_type='split', is_training_metric=True,
              lambda_l1=0.25485420011261123, lambda_l2=0.038225498964210035,
              learning_rate=0.1, max_bin=200, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=1000, n_jobs=6, num_leaf=28, num_leaves=31,
              objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
              silent=False, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0, verbose=2)

In [1199]:
light.fit(train[mlcol], train['EP_H_ND'])

LGBMRegressor(bagging_fraction=1.0, bagging_freq=4, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.7,
              importance_type='split', is_training_metric=True,
              lambda_l1=0.25485420011261123, lambda_l2=0.038225498964210035,
              learning_rate=0.1, max_bin=200, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=1000, n_jobs=6, num_leaf=28, num_leaves=31,
              objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
              silent=False, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0, verbose=2)

## Test the model

In [1200]:
y_pred = light.predict(test[mlcol])

In [1201]:
results_df = pd.DataFrame({'predicted' : y_pred, 'true' : test['EP_H_ND']})
results_df['error'] = np.abs((results_df['true'] - results_df['predicted'])/results_df['true'])*100

In [1202]:
# MAPE
results_df['error'].mean()

28.373972714041773

In [1203]:
# MAPE without >100%
results_df[results_df.error <= 100].error.mean()

23.56125297779272

In [1204]:
results_df

,predicted,true,error
6,87.307634,166.51,47.566132
12,87.603459,96.53,9.247427
15,71.703912,39.17,83.058238
19,63.247557,112.90,43.979135
25,117.063783,122.32,4.297104
...,...,...,...
1313,27.039075,70.71,61.760607
1332,115.529953,269.40,57.115830
1333,58.235143,67.02,13.107815
1338,37.702727,38.29,1.533751


# 1.2 inputs

## Random search

In [1205]:
mlcol = ['PERIODO_20', 'SUPERFICIE_LORDA', 'SUPERFICIE_NETTA',
           'VOLUME_LORDO', 
         'VOLUME_NETTO', 'SUPERFICIE_DISPERDENTE_x',
         'TRASMITTANZA_MEDIA_INVOLUCRO', 'TRASMITTANZA_MEDIA_COPERTURA',
         'TRASMITTANZA_MEDIA_BASAMENTO', 'TRASMITTANZA_MEDIA_SERRAMENTO',
         'dd','SUP_FINESTRATA','SUP_OPACA', 'SUPERFICIE_VETRATA_OPACA',
         'NUMERO_RICAMBI_ORARI', 'ALTEZZA_MEDIA_12'] # 'SUPERF_UTILE_RISCALDATA', 'VOLUME_LORDO_RISCALDATO', 

In [1206]:
cat_feats = ['PERIODO_20']

In [1207]:
param_dist = {'feature_fraction': np.linspace(0.4, 1, num=7),
              'num_leaf' : list(range(20,40)),
             'max_depth' : [-1],
             'max_bin' : [100, 200, 300, 500, 750, 1000, 2000],
             'bagging_fraction' : np.linspace(0.4, 1, num=14),
             'bagging_freq' : list(range(1,10)),
             'lambda_l1' : stats.uniform(0, 0.6),
             'lambda_l2' : stats.uniform(0, 0.6)}

lgbm = LGBMRegressor(n_estimators = 200, silent = True, verbose = 0, is_training_metric = True, n_jobs = 1, 
                     eval_metric  = 'mape')

n_iter_search = 100

random_search = RandomizedSearchCV(lgbm, param_distributions=param_dist, n_iter=n_iter_search, 
                                   scoring=neg_mape_scorer, cv = 5, n_jobs = 6, verbose = 2)

random_search.fit(cened[mlcol], cened['EP_H_ND'], categorical_feature = cat_feats)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    7.7s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:   36.8s
[Parallel(n_jobs=6)]: Done 353 tasks      | elapsed:  1.5min
[Parallel(n_jobs=6)]: Done 500 out of 500 | elapsed:  2.0min finished
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['PERIODO_20']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=LGBMRegressor(boosting_type='gbdt',
                                           class_weight=None,
                                           colsample_bytree=1.0,
                                           eval_metric='mape',
                                           importance_type='split',
                                           is_training_metric=True,
                                           learning_rate=0.1, max_depth=-1,
                                           min_child_samples=20,
                                           min_child_weight=0.001,
                                           min_split_gain=0.0, n_estimators=200,
                                           n_jobs=1, num_leaves=31,
                                           objective=None, random_sta...
                                        'lambda_l1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fcd677bc978>,
            

## Print results

In [1208]:
print(random_search.best_params_)

{'bagging_fraction': 0.8615384615384616, 'bagging_freq': 6, 'feature_fraction': 1.0, 'lambda_l1': 0.10885203763076472, 'lambda_l2': 0.49277472546241585, 'max_bin': 100, 'max_depth': -1, 'num_leaf': 21}


In [1209]:
print(random_search.best_score_)

-0.37566563364417754


In [1210]:
# Create a results df
cv_df = pd.DataFrame(random_search.cv_results_)
cv_df = cv_df.sort_values(by = ['rank_test_score']).reset_index()

# Filter columns
res_cols = [col for col in cv_df if col.startswith('split')]

# Select best iter
best_res = cv_df.loc[0]

# Print results for each fold
print(best_res[res_cols])

split0_test_score   -0.346799
split1_test_score   -0.391167
split2_test_score   -0.380053
split3_test_score   -0.361174
split4_test_score   -0.399135
Name: 0, dtype: object


## Train the final model

In [1211]:
light = LGBMRegressor(n_estimators = 1000, silent = False, verbose = 2, is_training_metric = True, n_jobs = 6)
light.set_params(**random_search.best_params_)

LGBMRegressor(bagging_fraction=0.8615384615384616, bagging_freq=6,
              boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              feature_fraction=1.0, importance_type='split',
              is_training_metric=True, lambda_l1=0.10885203763076472,
              lambda_l2=0.49277472546241585, learning_rate=0.1, max_bin=100,
              max_depth=-1, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=1000, n_jobs=6, num_leaf=21,
              num_leaves=31, objective=None, random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=False, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, verbose=2)

In [1212]:
light.fit(train[mlcol], train['EP_H_ND'])

LGBMRegressor(bagging_fraction=0.8615384615384616, bagging_freq=6,
              boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              feature_fraction=1.0, importance_type='split',
              is_training_metric=True, lambda_l1=0.10885203763076472,
              lambda_l2=0.49277472546241585, learning_rate=0.1, max_bin=100,
              max_depth=-1, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=1000, n_jobs=6, num_leaf=21,
              num_leaves=31, objective=None, random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=False, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, verbose=2)

## Test the model

In [1213]:
y_pred = light.predict(test[mlcol])

In [1214]:
results_df = pd.DataFrame({'predicted' : y_pred, 'true' : test['EP_H_ND']})
results_df['error'] = np.abs((results_df['true'] - results_df['predicted'])/results_df['true'])*100

In [1215]:
# MAPE
results_df['error'].mean()

37.41859109644729

In [1216]:
# MAPE without >100%
results_df[results_df.error <= 100].error.mean()

27.02407420534986

In [1217]:
results_df

,predicted,true,error
6,112.499463,166.51,32.436813
12,79.689516,96.53,17.445856
15,108.960902,39.17,178.174373
19,82.657732,112.90,26.786774
25,202.142740,122.32,65.257309
...,...,...,...
1313,117.221376,70.71,65.777649
1332,125.462242,269.40,53.429012
1333,105.769725,67.02,57.818151
1338,52.179849,38.29,36.275397
